In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! cp '/content/drive/MyDrive/KTH/Scalable ML/archive.zip' .
# ! unzip archive.zip && rm archive.zip

In [2]:
# ! pip install pytorch-lightning

In [3]:
import os
import torch
import random
import datetime
import numpy as np
import pandas as pd
import torch.nn as nn
import pytorch_lightning as pl

from PIL import Image
from torch.optim import AdamW
from torchvision import models
import torch.nn.functional as F
from argparse import ArgumentParser
from matplotlib import pyplot as plt
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

## Load and check raw data

In [4]:
folder = 'img_align_celeba/img_align_celeba'

imgs = [folder + "/" + f for f in sorted(folder)]
attrs = pd.read_csv('list_attr_celeba.csv').replace(-1, 0)

classes = attrs.columns
n_classes = len(classes) - 1
id2class = {i:classes[i+1] for i in range(n_classes)}

In [5]:

class CelebDataset(Dataset):
  def __init__(self,df,image_path,transform=None,mode='train'):
    super().__init__()
    self.attr=df.drop(['image_id'],axis=1)
    self.path=image_path
    self.image_id=df['image_id']
    self.transform=transform
    self.mode=mode
  
  def __len__(self):
    return self.image_id.shape[0]

  def __getitem__(self,idx:int):
    image_name=self.image_id.iloc[idx]
    image=Image.open(os.path.join(folder,image_name))
    attributes=np.asarray(self.attr.iloc[idx].T,dtype=np.float32)
    if self.transform:
      image=self.transform(image)
    return image, attributes    


# function to visualize dataset
def imshow(images,attr,idx:int):
    images=images.cpu().numpy().transpose((0,2,3,1))
    plt.imshow(images[idx] * std + mean)
    labels=attrs.columns.tolist()
    labels=labels[1:]
    att=attr[idx].numpy()
    labels=[label for label,a in list(zip(labels,att)) if a==1]
    plt.xlabel("\n".join(labels))
    plt.show()


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform=transforms.Compose([transforms.Resize((224,224)),
                  transforms.RandomVerticalFlip(p=0.5),
                  transforms.RandomHorizontalFlip(p=0.5),
                  transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=mean, std=std)])

valid_transform=transforms.Compose([transforms.Resize((224,224)),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=mean, std=std)])

In [6]:
train_df,test=train_test_split(attrs,test_size=0.1,shuffle=True,random_state=212)
valid_df,test_df=train_test_split(test,test_size=0.5,random_state=212)
 
train_data=CelebDataset(train_df,imgs,train_transform)
valid_data=CelebDataset(valid_df,imgs,valid_transform)
test_data= CelebDataset(test_df,imgs,valid_transform)

In [7]:
def calculate_metrics(output, target):
  """
  output: list of tensor from fc, [(B, H)]
  target: list of label [B]
  """
  from sklearn.metrics import accuracy_score

  predict = [torch.argmax(i, dim=1).cpu() for i in output]
  target = [i.cpu() for i in target]

  accs = [accuracy_score(y_true=t.numpy(), y_pred=p.numpy()) for t, p in zip(target, predict)]
  avg_acc = sum(accs) / len(accs)
  min_acc = min(accs)
  max_acc = max(accs)

  return avg_acc, min_acc, max_acc


class MultiBinMobileNet(pl.LightningModule):

  def __init__(self, n_classes):
    super().__init__()

    self.save_hyperparameters()
    self.n_classes = n_classes
    mnet = models.mobilenet_v2()
    
    # the input for the classifier should be two-dimensional, but we will have
    # [batch_size, channels, width, height]
    # so, let's do the spatial averaging: reduce width and height to 1
    self.pool = nn.AdaptiveAvgPool2d((1, 1))
    self.base_model = mnet.features
    self.fcs = [nn.Sequential(nn.Dropout(p=0.2), nn.Linear(in_features=mnet.last_channel, out_features=2)).cuda() for _ in range(n_classes)]

  def forward(self, x):
    x = self.base_model(x)
    x = self.pool(x)
    # reshape from [batch, channels, 1, 1] to [batch, channels] to put it into classifier
    x = torch.flatten(x, 1)

    return [fc(x) for fc in self.fcs]

  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=1e-3)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=1e-1, patience=2, verbose=True)
    return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}
    
  
  def training_step(self, train_batch, batch_idx):
    # Img [bsz, w, h, c]
    img, attrs = train_batch
    output = self.forward(img)
    attrs = torch.unbind(attrs, 1)

    train_loss = self.get_loss(output, attrs)
    return {'loss': train_loss}

  def validation_step(self, val_batch, batch_idx):
    img, attrs = val_batch
    output = self.forward(img)
    attrs = torch.unbind(attrs, 1)

    val_loss = self.get_loss(output, attrs)
    avg_acc, min_acc, max_acc = calculate_metrics(output, attrs)

    return {"val_loss": val_loss, 'avg_acc': avg_acc, 'min_acc': min_acc, 'max_acc': max_acc}

  def validation_epoch_end(self, outputs):
      """"""
      val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
      avg_acc = sum([x['avg_acc'] for x in outputs]) / len(outputs)
      max_acc = max([x['max_acc'] for x in outputs])
      min_acc = min([x['min_acc'] for x in outputs])
      self.log("val_loss", val_loss, prog_bar=True, logger=True)
      self.log("avg_acc", avg_acc, prog_bar=True, logger=True)
      self.log("max_acc", max_acc, prog_bar=True, logger=True)
      self.log("min_acc", min_acc, prog_bar=True, logger=True)

  def get_loss(self, output, truth):
    losses = sum([F.cross_entropy(output[i], truth[i].type(torch.LongTensor).cuda()) for i in range(self.n_classes)])
    return losses
  

In [ ]:
pl.seed_everything(1234)

batch_size = 32
n_epochs = 10
val_every_n_epoch = 1

train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=2)
valid_loader=DataLoader(valid_data,batch_size=batch_size,num_workers=2)
test_loader=DataLoader(test_data,batch_size=batch_size,num_workers=2)

model = MultiBinMobileNet(n_classes)

# Callbacks:
checkpoint = ModelCheckpoint(
    dirpath='models/ckpts/',
    filename="./fx-{epoch:02d}-{val_loss:.7f}",
    monitor="val_loss"
)

earlystopping = EarlyStopping(monitor='val_loss',
                              min_delta=0.01,
                              patience=5,
                              verbose=False,
                              mode="min")

trainer = pl.Trainer(
    gpus=1,
    # limit_train_batches=100,
    max_epochs=n_epochs,
    enable_checkpointing=True,
    check_val_every_n_epoch=val_every_n_epoch,
    callbacks=[checkpoint, earlystopping, LearningRateMonitor()]
)

trainer.fit(model, train_loader, valid_loader)

In [14]:
model = MultiBinMobileNet.load_from_checkpoint('/content/models/ckpts/fx-epoch=00-val_loss=9.8684387.ckpt')

# use model.forward() to get prediction


# convert from id to names using dict

id2class
